## Juliaでpythonのパッケージ(sdv)を動かしてみる
sdvは完全に独自パッケージなので、できるかどうか  
参考: https://qiita.com/Ag_smith/items/0352890b8cf3420765d0  
https://qiita.com/Miyazaki-Yu/items/51d1e29b1df5ce29bba4  
https://gist.github.com/terasakisatoshi/23b7b69af2893691a992fb7263e185c9  
https://twitter.com/genkuroki/status/1307386185942069248?s=20&t=IQHHqCVKwj0EseVkjN_MdA  
https://qiita.com/Ken-Kuroki/items/115d0679fdce1d15b9d1  

In [1]:
# 初回のインストール
using Pkg
Pkg.add("PyCall")
Pkg.add("Conda")
Pkg.add("Pandas")
Pkg.add("DataFrames")
Pkg.add("CSV")

using PyCall
using Conda
Conda.update()
Conda.add("numpy")
Conda.add("scikit-learn")
Conda.add("pandas")
Conda.add("sdv")

ENV["PYTHON"] = ENV["HOME"] * "/.julia/conda/3/bin/python3.10"
Pkg.build("PyCall")


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
[ Info: Running `conda update -y --all conda` in root environment


Solving environment: ...working... done

# All requested packages already installed.



[ Info: Running `conda install -y numpy` in root environment


Solving environment: ...working... done

# All requested packages already installed.



[ Info: Running `conda install -y scikit-learn` in root environment


...

[ Info: Running `conda install -y pandas` in root environment


Solving environment: ...working... done

# All requested packages already installed.



[ Info: Running `conda install -y sdv` in root environment


Solving environment: ...working... done

# All requested packages already installed.



    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/6e47d11ea2776bc5627421d59cdcc1296c058071/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/62f417f6ad727987c755549e9cd88c46578da562/build.log`


In [2]:
using PyCall
using CSV
using DataFrames
using Pandas: Pandas

In [4]:
pandas = pyimport("pandas")
pydf = pandas.read_csv("adult.csv")

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39.0,7,77516.0,9,13.0,4,1,1,4,1,2174.0,0.0,40.0,39,1
1,50.0,6,83311.0,9,13.0,2,4,0,4,1,0.0,0.0,13.0,39,1
2,38.0,4,215646.0,11,9.0,0,6,1,4,1,0.0,0.0,40.0,39,1
3,53.0,4,234721.0,1,7.0,2,6,0,2,1,0.0,0.0,40.0,39,1
4,28.0,4,338409.0,9,13.0,2,10,5,2,0,0.0,0.0,40.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39.0,4,215419.0,9,13.0,0,10,1,4,0,0.0,0.0,36.0,39,1
48838,64.0,0,321403.0,11,9.0,6,0,2,2,1,0.0,0.0,40.0,39,1
48839,38.0,4,374983.0,9,13.0,2,10,0,4,1,0.0,0.0,50.0,39,1
48840,44.0,4,83891.0,9,13.0,0,1,3,1,1,5455.0,0.0,40.0,39,1


In [5]:
py"""
from sdv.tabular import GaussianCopula
def make_model(df):
    gc = GaussianCopula()
    gc.fit(df)
    return gc
"""
gc_model = py"make_model"(pydf)

PyObject <sdv.tabular.copulas.GaussianCopula object at 0x2abdc35e0>

In [7]:
py"""
def sample(model, sample_num):
    new_data = model.sample(sample_num)
    return new_data
"""
new_data = py"sample"(gc_model,1000)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,54.0,6,129929.0,13,14.0,0,13,2,3,1,3522.0,538.0,34.0,35,1
1,32.0,5,185937.0,14,13.0,1,3,2,3,0,2642.0,174.0,39.0,35,1
2,24.0,2,346743.0,13,9.0,2,0,5,3,0,7932.0,129.0,33.0,33,1
3,43.0,2,61057.0,14,11.0,3,6,2,2,1,6679.0,201.0,35.0,29,1
4,41.0,5,212028.0,2,5.0,2,6,1,2,1,8150.0,144.0,50.0,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,43.0,2,101189.0,8,13.0,0,7,0,3,1,4656.0,49.0,30.0,34,1
996,36.0,5,230498.0,8,9.0,1,13,1,4,1,6717.0,82.0,51.0,33,1
997,18.0,5,62133.0,5,10.0,1,12,4,3,1,2632.0,508.0,28.0,15,0
998,24.0,4,122254.0,12,11.0,6,9,1,4,0,476.0,457.0,49.0,36,0
